# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [13]:
!cat publications_nk.tsv

pub_date	title	venue	excerpt	citation	url_slug	paper_url	slides_url
03.02.25	Deep learning adaptive Model Predictive Control of Fed-Batch Cultivations	arXiv preprint	Bioprocesses are often characterised by nonlinear and uncertain dynamics, posing particular challenges for model predictive control (MPC) algorithms due to their computational demands when applied to nonlinear systems. Recent advances in optimal control theory have demonstrated that concepts from convex optimisation, tube MPC, and differences of convex functions (DC) enable efficient, robust online process control. Our approach is based on DC decompositions of nonlinear dynamics and successive linearisations around predicted trajectories. By convexity, the linearisation errors have tight bounds and can be treated as bounded disturbances within a robust tube MPC framework. We describe a systematic, data-driven method for computing DC model representations using deep learning neural networks with a special convex structure, 

## Import pandas

We are using the very handy pandas library for dataframes.

In [14]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [16]:
publications = pd.read_csv("publications_nk.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url,slides_url
0,03.02.25,Deep learning adaptive Model Predictive Contro...,arXiv preprint,Bioprocesses are often characterised by nonlin...,"Krausch, N., Doff-Sotta, M., Cannon, M., Neuba...",paper-title-number-1,https://arxiv.org/abs/2502.01488,NaN
1,04.01.24,Recombinant protein expression in proteome-red...,Biotechnology and Bioengineering,NaN,"Lara, A. R., Utrilla, J., Martínez, L. M., Kra...",paper-title-number-2,https://analyticalsciencejournals.onlinelibrar...,NaN
2,02.04.24,Hochdurchsatz-Strategien für modellbasierte Bi...,BIOspektrum,NaN,"Cai, L., Kemmer, A., Krausch, N., Neubauer, P....",paper-title-number-3,https://link.springer.com/article/10.1007/s122...,NaN
3,26.06.24,Handling nonlinearities and uncertainties of f...,Computer Aided Chemical Engineering,NaN,"Krausch, N., Doff-Sotta, M., Canon, M., Neubau...",paper-title-number-4,https://www.sciencedirect.com/science/article/...,NaN
4,05.07.23,Model-Based Characterization of E. coli Strai...,Bioengineering,NaN,"Krausch, N., Kaspersetz, L., Gayt�n-Castro, R....",paper-title-number-5,https://www.mdpi.com/2306-5354/10/7/808,NaN
5,15.02.23,Model predictive control and moving horizon es...,Computers & Chemical Engineering,NaN,"Kim, J. W., Krausch, N., Aizpuru, J., Barz, T....",paper-title-number-6,https://www.sciencedirect.com/science/article/...,NaN
6,23.03.23,Production of modified nucleosides in a contin...,International Journal of Molecular Sciences,NaN,"Thiele, I., Yehia, H., Krausch, N., Birkholz, ...",paper-title-number-7,https://www.mdpi.com/1422-0067/24/7/6081,NaN
7,01.12.22,High_throughput screening of optimal process c...,Biotechnology and bioengineering,NaN,"Krausch, N., Kim, J. W., Barz, T., Lucia, S., ...",paper-title-number-8,https://analyticalsciencejournals.onlinelibrar...,NaN
8,01.01.22,Model predictive control guided with optimal e...,IFAC-PapersOnLine,NaN,"Kim, J. W., Krausch, N., Aizpuru, J., Barz, T....",paper-title-number-9,https://www.sciencedirect.com/science/article/...,NaN
9,01.01.22,Optimal operation of parallel mini-bioreactors...,Computer aided chemical engineering,NaN,"Krausch, N., Kim, J. W., Lucia, S., Gro�, S., ...",paper-title-number-10,https://www.sciencedirect.com/science/article/...,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [17]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [18]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.slides_url)) > 5:
        md += "\nslidesurl: '" + item.slides_url + "'"

    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"

    if len(str(item.slides_url)) > 5:
        md += "\n[Download slides here](" + item.slides_url + ")\n" 

    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [19]:
!ls ../_publications/

01.01.20-paper-title-number-14.md 11.11.20-paper-title-number-11.md
01.01.22-paper-title-number-10.md 14.05.20-paper-title-number-13.md
01.01.22-paper-title-number-9.md  14.09.20-paper-title-number-12.md
01.11.19-paper-title-number-16.md 15.02.23-paper-title-number-6.md
01.12.19-paper-title-number-15.md 15.07.19-paper-title-number-17.md
01.12.22-paper-title-number-8.md  19.06.19-paper-title-number-18.md
02.04.24-paper-title-number-3.md  21.11.18-paper-title-number-20.md
03.02.25-paper-title-number-1.md  23.03.23-paper-title-number-7.md
04.01.24-paper-title-number-2.md  24.05.19-paper-title-number-19.md
05.07.23-paper-title-number-5.md  26.06.24-paper-title-number-4.md


In [16]:
!cat ../_publications/2009-10-01-paper-title-number-1.md

---
title: "Paper Title Number 1"
collection: publications
permalink: /publication/2009-10-01-paper-title-number-1
excerpt: 'This paper is about the number 1. The number 2 is left for future work.'
date: 2009-10-01
venue: 'Journal 1'
slidesurl: 'http://academicpages.github.io/files/slides1.pdf'
paperurl: 'http://academicpages.github.io/files/paper1.pdf'
citation: 'Your Name, You. (2009). &quot;Paper Title Number 1.&quot; <i>Journal 1</i>. 1(1).'
---
This paper is about the number 1. The number 2 is left for future work.

[Download slides here](http://academicpages.github.io/files/slides1.pdf)

[Download paper here](http://academicpages.github.io/files/paper1.pdf)

Recommended citation: Your Name, You. (2009). "Paper Title Number 1." <i>Journal 1</i>. 1(1).